# 위쪽 부분은 TEST, 아래쪽이 함수화한 부분입니다!

# SEAT 처리 : 완료

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
import re

plt.rc('font', family = 'Malgun Gothic')
filterwarnings('ignore')

# 전체 데이터 출력 및 확인 옵션 설정
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_info_columns

100

In [2]:
# place에 따라 자리가 다를 것으로 생각하고, 나눠서 작업하기
df = pd.read_csv("dataset/2023bigcontest_advanced_sac.csv")
df['place'].unique()

array(['콘서트홀', '리사이틀홀', 'IBK챔버홀'], dtype=object)

In [3]:
df_concert = df.query("`place`=='콘서트홀'")
df_chamber = df.query("`place`=='IBK챔버홀'")
df_recital = df.query("`place`=='리사이틀홀'")

In [4]:
df_concert.seat.sample(10)
# 3층 N블록1열 9 | 합창석 F블록4열 7 | 2층 BOX1 4 와 같이 split으로 3묶음으로 나뉘며, '블록'을 기준으로 나누어 담으면 될듯함. BOX는 '블록'으로 취급하기로 함.

83796        3층 D블록4열 7
1119830      1층 B블록2열 7
750682       2층 D블록5열 3
1331247     1층 E블록13열 5
1365142     2층 B블록4열 11
53802      합창석 G블록2열 18
1355096     2층 E블록3열 11
1088749     2층 E블록5열 18
729768       1층 D블록6열 7
1234835     1층 D블록15열 2
Name: seat, dtype: object

In [5]:
# 'seat_floor', 'seat_block', 'seat_col', 'seat_num' 생성
df_concert1 = df_concert.copy() #실험용

# 1.seat_floor 생성
df_concert1['seat_floor'] = df_concert1['seat'].str.split(" ").str[0]
df_concert1['seat_floor'] = df_concert1['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제
df_concert1['seat_floor'] = df_concert1['seat_floor'].replace("합창석",4,regex=True) # 3층까지만 있으므로, 합창석은 4로 대체
df_concert1['seat_floor'].astype(int)

# 2. seat_block, seat_col 생성 
# 2-1. BOX1과 같이 "블록"으로 split 불가한 경우 대비 : seat_block에 넣어두고, 이 경우 seat_col은 nan이 되므로 -1 부여
df_concert1['seat_block'] = df_concert1['seat'].str.split(" ").str[1]
df_concert1['seat_col'] = -1

# 2-2. box df, block df 두 가지로 구분
df_concert_box = df_concert1.query('not seat_block.str.contains("블록")')
df_concert_block = df_concert1.query('seat_block.str.contains("블록")')

# 2-3. block df에서 "블록"기준으로 구분하여 담고, "열"글자 삭제
df_concert_block[['seat_block', 'seat_col']] = df_concert_block['seat_block'].str.split("블록", expand=True)
df_concert_block['seat_col'] = df_concert_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

df_concert2 = pd.concat([df_concert_box, df_concert_block])
df_concert2['seat_col'].astype(int)    
df_concert2

# 3. seat_num 생성
df_concert2['seat_num'] = df_concert2['seat'].str.split(" ").str[-1]
df_concert2['seat_num'].astype(int)

0          10
11          5
13          3
18          2
98          5
           ..
1920853     2
1920855     4
1920856    12
1920858     8
1920864     4
Name: seat_num, Length: 1096968, dtype: int64

In [6]:
df_chamber.seat.sample(10)
# 1층 B블록 1열 1 | 2층 BOX2 3 과 같음. 공백 기준 split으로 나눠담기.

1906075        2층 BOX1 1
930746     1층 B블록 13열 10
172010       1층 B블록 9열 7
1810301     1층 B블록 14열 4
1025837     1층 B블록 10열 6
75636       1층 B블록 6열 12
1772609     1층 C블록 14열 2
512058     1층 B블록 14열 14
1669221      1층 B블록 5열 4
1840576     1층 B블록 7열 10
Name: seat, dtype: object

In [7]:
# 'seat_floor', 'seat_block', 'seat_col', 'seat_num' 생성
df_chamber1 = df_chamber.copy() #실험용

# 1.seat_floor 생성
df_chamber1['seat_floor'] = df_chamber1['seat'].str.split(" ").str[0]
df_chamber1['seat_floor'] = df_chamber1['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제. 1층과 2층만 있음
df_chamber1['seat_floor'].astype(int)

# 2. seat_block 생성 
df_chamber1['seat_block'] = df_chamber1['seat'].str.split(" ").str[1]

# 3. seat_col 생성 :  box df, block df 두 가지로 구분
df_chamber_box = df_chamber1.query('not seat_block.str.contains("블록")')

# 3-1. BOX석이 seat_block으로 입력될 때 seat_col은 nan이 되므로 -1 부여
df_chamber_box['seat_col'] = -1

# 3-2. 블록이 있으면 구분되어 있으므로 데이터 부여
df_chamber_block = df_chamber1.query('seat_block.str.contains("블록")')

# 3-3. seat 데이터를 이용해 공백 기준으로 구분하여 담고, "블록"&"열"글자 삭제
df_chamber_block['seat_col'] = df_chamber_block['seat'].str.split(" ").str[2]

df_chamber_block['seat_block'] = df_chamber_block['seat_block'].replace("블록$","",regex=True) # 마지막 글자가 '블록'이면 삭제
df_chamber_block['seat_col'] = df_chamber_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

df_chamber2 = pd.concat([df_chamber_box, df_chamber_block])
df_chamber2['seat_col'].astype(int)    

# 3. seat_num 생성
df_chamber2['seat_num'] = df_chamber2['seat'].str.split(" ").str[-1]
df_chamber2['seat_num'].astype(int)

136         8
312        10
330         5
1230        6
1351       10
           ..
1920860     9
1920861     2
1920863     6
1920866     5
1920867     4
Name: seat_num, Length: 477410, dtype: int64

In [8]:
df_recital.seat.sample(10)
#  1층 6열 2 |  1층 BOX1 2열 23 | 1층 우측 1열 7 | 2층 좌측 1열 9 와 같음.

646570           1층 6열 6
687712          2층 2열 10
1123588    1층 BOX2 2열 21
486264           1층 7열 2
14649      2층 BOX3 1열 10
1070931     2층 BOX3 1열 5
139279           1층 8열 2
430105         1층 11열 17
1876277    1층 BOX2 1열 17
514524           1층 1열 8
Name: seat, dtype: object

In [9]:
# 'seat_floor', 'seat_block', 'seat_col', 'seat_num' 생성
df_recital1 = df_recital.copy() #실험용

# 1.seat_floor 생성
df_recital1['seat_floor'] = df_recital1['seat'].str.split(" ").str[0]
df_recital1['seat_floor'] = df_recital1['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제. 1층과 2층만 있음
df_recital1['seat_floor'].astype(int)

# 2. seat_block 생성 : block, noblock으로 구분 : split list의 길이가 3이면 nan 값이므로 -1 부여
df_recital_noblock = df_recital1[df_recital1['seat'].str.split(" ").str.len() < 4]
df_recital_noblock['seat_block'] = -1

df_recital_block = df_recital1[df_recital1['seat'].str.split(" ").str.len() == 4]
df_recital_block['seat_block'] = df_recital_block['seat'].str.split(" ").str[1]

df_recital2 = pd.concat([df_recital_noblock, df_recital_block])

# 3. seat_col 생성 : 뒤에서 두 번째 값으로 부여
df_recital2['seat_col'] = df_recital2['seat'].str.split(" ").str[-2]
df_recital2['seat_col'] = df_recital2['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제
df_recital2['seat_col'].astype(int)    

# 3. seat_num 생성
df_recital2['seat_num'] = df_recital2['seat'].str.split(" ").str[-1]
df_recital2['seat_num'].astype(int)

1           5
3           3
14          5
23          9
25          1
           ..
1920719    12
1920778    12
1920786     9
1920822     7
1920857     7
Name: seat_num, Length: 346490, dtype: int64

# 함수화 : sep_seat

In [10]:
def sep_seat(df):
    # 0. place별로 df 구분
    df_concert = df.query("`place`=='콘서트홀'")
    df_chamber = df.query("`place`=='IBK챔버홀'")
    df_recital = df.query("`place`=='리사이틀홀'")


    # ===concert 홀 처리===
    # 1.seat_floor 생성
    df_concert['seat_floor'] = df_concert['seat'].str.split(" ").str[0]
    df_concert['seat_floor'] = df_concert['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제
    df_concert['seat_floor'] = df_concert['seat_floor'].replace("합창석",4,regex=True) # 3층까지만 있으므로, 합창석은 4로 대체

    # 2. seat_block, seat_col 생성 
    # 2-1. BOX1과 같이 "블록"으로 split 불가한 경우 대비 : seat_block에 넣어두고, 이 경우 seat_col은 nan이 되므로 -1 부여
    df_concert['seat_block'] = df_concert['seat'].str.split(" ").str[1]
    df_concert['seat_col'] = -1

    # 2-2. box df, block df 두 가지로 구분
    df_concert_box = df_concert.query('not seat_block.str.contains("블록")')
    df_concert_block = df_concert.query('seat_block.str.contains("블록")')

    # 2-3. block df에서 "블록"기준으로 구분하여 담고, "열"글자 삭제
    df_concert_block[['seat_block', 'seat_col']] = df_concert_block['seat_block'].str.split("블록", expand=True)
    df_concert_block['seat_col'] = df_concert_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

    df_concert_new = pd.concat([df_concert_box, df_concert_block])

    # 3. seat_num 생성
    df_concert_new['seat_num'] = df_concert_new['seat'].str.split(" ").str[-1]


    # ===chamber 홀 처리===
    # 1.seat_floor 생성
    df_chamber['seat_floor'] = df_chamber['seat'].str.split(" ").str[0]
    df_chamber['seat_floor'] = df_chamber['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제. 1층과 2층만 있음

    # 2. seat_block 생성 
    df_chamber['seat_block'] = df_chamber['seat'].str.split(" ").str[1]

    # 3. seat_col 생성 :  box df, block df 두 가지로 구분
    df_chamber_box = df_chamber.query('not seat_block.str.contains("블록")')
    # 3-1. BOX석이 seat_block으로 입력될 때 seat_col은 nan이 되므로 -1 부여
    df_chamber_box['seat_col'] = -1

    # 3-2. 블록이 있으면 구분되어 있으므로 데이터 부여
    df_chamber_block = df_chamber.query('seat_block.str.contains("블록")')

    # 3-3. seat 데이터를 이용해 공백 기준으로 구분하여 담고, "블록"&"열"글자 삭제
    df_chamber_block['seat_col'] = df_chamber_block['seat'].str.split(" ").str[2]
    df_chamber_block['seat_block'] = df_chamber_block['seat_block'].replace("블록$","",regex=True) # 마지막 글자가 '블록'이면 삭제
    df_chamber_block['seat_col'] = df_chamber_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

    df_chamber_new = pd.concat([df_chamber_box, df_chamber_block])

    # 4. seat_num 생성
    df_chamber_new['seat_num'] = df_chamber_new['seat'].str.split(" ").str[-1]

    # ===recital 홀 처리===
    # 1.seat_floor 생성
    df_recital['seat_floor'] = df_recital['seat'].str.split(" ").str[0]
    df_recital['seat_floor'] = df_recital['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제. 1층과 2층만 있음

    # 2. seat_block 생성 : block, noblock으로 구분 : split list의 길이가 3이면 nan 값이므로 -1 부여
    df_recital_noblock = df_recital[df_recital['seat'].str.split(" ").str.len() < 4]
    df_recital_noblock['seat_block'] = -1

    df_recital_block = df_recital[df_recital['seat'].str.split(" ").str.len() == 4]
    df_recital_block['seat_block'] = df_recital_block['seat'].str.split(" ").str[1]

    df_recital_new = pd.concat([df_recital_noblock, df_recital_block])

    # 3. seat_col 생성 : 뒤에서 두 번째 값으로 부여
    df_recital_new['seat_col'] = df_recital_new['seat'].str.split(" ").str[-2]
    df_recital_new['seat_col'] = df_recital_new['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

    # 3. seat_num 생성
    df_recital_new['seat_num'] = df_recital_new['seat'].str.split(" ").str[-1]

    
    df_new = pd.concat([df_concert_new, df_chamber_new, df_recital_new])
    df_new[['seat_floor', 'seat_col', 'seat_num']] = df_new[['seat_floor', 'seat_col', 'seat_num']].astype(int)
    result = df_new.drop(['seat'], axis=1)
    return result.sort_index()

In [11]:
# 블록은 BOXn, A,B,C, 좌측, 우측 이런 식으로 되어 있음. 임의의 int 값을 부여해야하지 않을지? 추후 논의.

# DISCOUNT 처리 : 완료

In [12]:
# discount는 %가 포함된 것과, 아닌 것으로 분류
df_rate = df.query('discount_type.str.contains("%")')
df_norate = df.query('not discount_type.str.contains("%")')

In [19]:
df['discount_type'].unique()

array(['일반', '초대권', '블루회원 할인20%', '골드회원 할인20%', '기획사',
       'K-lang 멤버십 회원(1인2매)20%', '기획사판매', '골드회원 할인30%', 'R석 기획사 할인',
       '싹틔우미 할인40%', '라흐마니노프탄생150주년(8매/3.27까지)20%', '공연진행석', '골드회원 할인10%',
       '장애인/국가유공자 할인50%', '싹딜가(골드)50%', '당일할인티켓', '그린회원 할인5%',
       '초,중,고,대학생(본인만)30%', '조기예매 할인40%', '그린회원 할인20%', '한화초대석',
       '단체100인이상50%', '공연예매권', '초/중/고등학생 할인(본인만)30%', '골드회원 할인15%',
       '골드회원 할인25%', '우리카드 우수회원', '조기예매 할인(1인8매/3월11일까지)30%', '신세계석',
       '골드회원 할인5%', '정기회원', '만65세 이상(본인만)50%', '싹딜가(블루)50%', '그린회원 할인15%',
       '직장인 할인(1인2매)20%', '블루회원 할인10%', '기획사판매20%',
       '초/중/고/대학생 할인(본인만)50%', '문화릴레이(1인2매/공연유료티켓제시)10%', '조기예매 할인10%',
       '노블회원 할인40%', 'K-Lang 할인(1인2매)20%', '골드회원 할인40%',
       '초/중/고등학생 할인(본인만)20%', '조기예매 할인(1인2매/~10월31일까지)30%', '기업석',
       '초/중/고/대학생 할인30%', '초/중/고/대학생 할인50%', '의사상자(동반1인/신분증지참)50%',
       '초,중,고,대학생(본인만)50%', '조기예매할인(1인8매/~6.30까지)30%', '그린회원 할인10%',
       '블루회원 할인5%', '연주자 할인50%', '스탭 할인30%', '홍보진행',
       'KOSYMI  유료회원(1

In [14]:
# discount_type 중에서 "~%할인", "~할인50%_"로 끝나는 데이터가 있으므로 미리 처리
df_rate['discount_type'] = df_rate['discount_type'].replace("_$","",regex=True) 
df_rate['discount_type'] = df_rate['discount_type'].replace("할인$","",regex=True) 
# 할인 이름과 할인율 추출
df_rate['discount_name'] = df_rate['discount_type'].apply(lambda x: re.sub(r'(\d+)%$', '', x))
df_rate['discount_rate'] = df_rate['discount_type'].str.extract(r'(\d+)%$')

In [15]:
df_norate['discount_type'].unique()
# price = 0이면 : 100 (초대권인데 돈을 낸 경우도 있고 하므로)
# 일반 : 0
# 그 외에는 정확한 할인율을 알 수 없기 때문에 -1로 처리하겠음.

df100 = df_norate[df_norate['price'] == 0]
df100['discount_name'] = df100['discount_type']
df100['discount_rate'] = 100

df100x = df_norate[df_norate['price'] != 0]
df0 = df100x[df100x['discount_type'] == "일반"]
df0['discount_name'] = "일반"
df0['discount_rate'] = 0

dfetc = df100x[df100x['discount_type'] != "일반"]
dfetc['discount_name'] = dfetc['discount_type']
dfetc['discount_rate'] = -1

df_new = pd.concat([df100, df0, dfetc])
df_new[['price','discount_type','discount_name','discount_rate']].sample(10)

,price,discount_type,discount_name,discount_rate
950407,0,초대권,초대권,100
1458674,0,초대권,초대권,100
1191833,0,초대권,초대권,100
61530,0,초대권,초대권,100
48117,0,초대권,초대권,100
1013849,0,초대권,초대권,100
611307,0,초대권,초대권,100
1148320,20000,일반,일반,0
650579,0,초대권,초대권,100
361393,30000,일반,일반,0


In [25]:
df_new[['price','discount_type','discount_name','discount_rate']].sample(10)

,price,discount_type,discount_name,discount_rate
1185299,0,초대권,초대권,100
108044,0,초대권,초대권,100
682456,0,기획사판매,기획사판매,100
1275079,0,초대권,초대권,100
1485000,0,한화초대석,한화초대석,100
759408,0,초대권,초대권,100
287457,0,초대권,초대권,100
1562750,0,초대권,초대권,100
253349,0,초대권,초대권,100
513488,0,초대권,초대권,100


# 함수화 : sep_discount(df):

In [99]:
def sep_discount(df):
    # 1. %가 포함된 것과, 아닌 것으로 분류
    df_plain = df.query('not discount_type.str.contains("%")')
    df_rate = df.query('discount_type.str.contains("%")')
    
    # 2. %가 포함되어 있지 않은 df_plain 처리
    # if price == 0 : 100 (초대권인데 돈을 낸 경우도 존재)
    # if price != 0 : '일반'이면 0, 그 외에는 할인율을 알 수 없기 때문에 -1로 처리
    df_plain100 = df_plain[df_plain['price'] == 0]
    df_plain100['discount_name'] = df_plain100['discount_type']
    df_plain100['discount_rate'] = 100

    df_plain100x = df_plain[df_plain['price'] != 0]
    df_plain100x = df_plain100x[df_plain100x['discount_type'] == "일반"]
    df_plain100x['discount_name'] = "일반"
    df_plain100x['discount_rate'] = 0

    df_etc = df_plain100x[df_plain100x['discount_type'] != "일반"]
    df_etc['discount_name'] = df_etc['discount_type']
    df_etc['discount_rate'] = -1 

    # 3. %가 포함된 df_rate 처리
    # if price == 0 : df_rate에서도 가격이 0이라면 discount_rate는 100이므로, % 데이터를 무시하고 100으로 처리.
    # if price != 0 : "~%할인", "~%_"로 끝나는 데이터 처리 후, 할인 이름과 할인률을 추출.
    df_rate100 = df_rate[df_rate['price'] == 0]
    df_rate100['discount_name'] = df_rate100['discount_type']
    df_rate100['discount_rate'] = 100

    df_rate100x = df_rate[df_rate['price'] != 0]    
    df_rate100x['discount_type'] = df_rate100x['discount_type'].replace("_$","",regex=True) 
    df_rate100x['discount_type'] = df_rate100x['discount_type'].replace("할인$","",regex=True) 
    df_rate100x['discount_name'] = df_rate100x['discount_type'].apply(lambda x: re.sub(r'(\d+)%$', '', x))
    df_rate100x['discount_rate'] = df_rate100x['discount_type'].str.extract(r'(\d+)%$')

    # 4. df_new 생성, data type 통일
    df_new = pd.concat([df_plain100, df_plain100x, df_etc, df_rate100, df_rate100x])
    df_new['discount_rate'] = df_new['discount_rate'].astype(int)
    result = df_new.drop(['discount_type'], axis=1)
    return result.sort_index()

In [100]:
# 함수 잘 들어갔는지 test
df1 = pd.read_csv("dataset/2023bigcontest_advanced_sac.csv")
df2 = sep_seat(df1)
df3 = sep_discount(df2)

df3.isnull().sum()

age                  1293202
gender               1292791
membership_type_1    1292791
membership_type_2    1423744
membership_type_3    1731947
membership_type_4    1871398
membership_type_5    1899895
membership_type_6    1901145
tran_date                  0
tran_time                  0
play_date                  0
play_st_time               0
price                      0
ticket_cancel              0
performance_code           0
pre_open_date         856265
open_date                340
genre                  25574
place                      0
running_time               0
intermission               0
member_yn                  0
seat_floor                 0
seat_block                 0
seat_col                   0
seat_num                   0
discount_name              0
discount_rate              0
dtype: int64

In [101]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1901145 entries, 0 to 1920867
Data columns (total 28 columns):
 #   Column             Dtype  
---  ------             -----  
 0   age                float64
 1   gender             object 
 2   membership_type_1  object 
 3   membership_type_2  object 
 4   membership_type_3  object 
 5   membership_type_4  object 
 6   membership_type_5  object 
 7   membership_type_6  float64
 8   tran_date          int64  
 9   tran_time          int64  
 10  play_date          int64  
 11  play_st_time       int64  
 12  price              int64  
 13  ticket_cancel      int64  
 14  performance_code   int64  
 15  pre_open_date      float64
 16  open_date          float64
 17  genre              object 
 18  place              object 
 19  running_time       int64  
 20  intermission       int64  
 21  member_yn          object 
 22  seat_floor         int64  
 23  seat_block         object 
 24  seat_col           int64  
 25  seat_num           int6

In [108]:
df3[['price','discount_name','discount_rate']].sample(10)

,price,discount_name,discount_rate
736479,0,초대권,100
4233,0,기획사판매,100
1785448,54000,조기예매 할인,10
234977,0,기획사,100
315248,0,초대권,100
160137,0,초대권,100
2521,162000,골드회원 할인,10
517527,80000,블루회원 할인,20
600007,90000,블루회원 할인,10
314766,10000,일반,0
